<a href="https://colab.research.google.com/github/tkg31302/Algo-trading/blob/main/Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bernoulli Naive Bayes

In [11]:
# import Apple data
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
data = yf.download('AAPL','2010-01-1', '2019-01-01')
data.index = pd.to_datetime(data.index)
data.tail(3)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-12-27,38.959999,39.192501,37.517502,39.037498,37.517975,212468400
2018-12-28,39.375000,39.630001,38.637501,39.057499,37.537197,169165600
2018-12-31,39.632500,39.840000,39.119999,39.435001,37.900005,140014000


In [12]:
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py310hde88566_4.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.10/dist-packages/ lib/python3.10/site-packages/talib --strip-components=3
import talib

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4295    0  4295    0     0  31235      0 --:--:-- --:--:-- --:--:-- 31350
100  517k  100  517k    0     0  1742k      0 --:--:-- --:--:-- --:--:-- 1742k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4283    0  4283    0     0  26191      0 --:--:-- --:--:-- --:--:-- 26276
100  392k  100  392k    0     0  1457k      0 --:--:-- --:--:-- --:--:-- 1457k


In [13]:
import talib as ta
data['RSI'] = ta.RSI(data['Close'],14)
data.loc[:, 'RSI_signal'] = np.where(data['RSI'] < 40, 1, 0)
data['slowk'], data['slowd'] = ta.STOCH(data['High'], data['Low'], data['Adj Close'], fastk_period=14, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
data.loc[:, 'Stoch_signal'] = np.where(data['slowk'] > data['slowd'], 1, 0)

In [14]:
data['ret1'] = data['Adj Close'].pct_change()
data['retFut1'] = data.ret1.shift(-1)

In [15]:
# Define predictor variables (X) and a target variable (y)
data = data.dropna()
predictor_list = ['RSI_signal','Stoch_signal']
X = data[predictor_list]

import numpy as np
y = np.where(data.retFut1 > 0.0, 1.0, 0)

In [16]:
# Split the data into train and test dataset
from sklearn.model_selection import train_test_split

train_length = int(len(data)*0.80)

X_train = X[:train_length]
X_test =  X[train_length:]

y_train = y[:train_length]
y_test = y[train_length:]

In [17]:
from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB().fit(X_train, y_train)
# Fit the model on train dataset
model.fit(X_train, y_train)
predicted_signal = model.predict(X_test)

In [18]:
from sklearn import metrics
print("Binomial Naive Bayes model accuracy(in %):", metrics.accuracy_score(y_test, predicted_signal)*100)

Binomial Naive Bayes model accuracy(in %): 51.77777777777778
